## Домашнее задание к лекции "Рекуррентные сети" 

 
### Задание 1.
#### Обучите нейронную сеть решать шифр цезаря.
##### Что надо сделать:
* 1.Написать алгоритм шифра Цезаря для генерации выборки (сдвиг на К каждой буквы. Например, при сдвиге на 2 буква “А” переходит в букву “В” и тп)
* 2.Сделать нейронную сеть
* 3.Обучить ее (вход - зашифрованная фраза, выход - дешифрованная фраза)
* 4.Проверить качество

In [1]:
import torch
from torch import nn as nn
import pandas as pd
import numpy as np
import time

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [3]:
CHAR = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ'
INDEX_TO_CHAR = ['none'] + [w for w in CHAR]
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}

In [43]:
text = pd.read_csv('onegin.txt', sep="\s*\|\s*", skiprows=[1], engine='python').dropna(how="all", axis=1)  
phrases = text['Евгений Онегин'].tolist()
text_ = [i for i in phrases]
text_train = []
for k in text_:  
    text_train += k

text_train = [k for k in text_train if k in INDEX_TO_CHAR]
print(text_train[:21])


['К', 'л', 'а', 'с', 'с', 'и', 'к', 'а', 'д', 'л', 'я', 'ш', 'к', 'о', 'л', 'ь', 'н', 'и', 'к', 'о', 'в']


In [44]:
MAX_LEN = 35
Y = torch.zeros((len(text_train)), dtype=int)
Y.shape

torch.Size([31345])

In [45]:
for i in range(len(text_train)):
    for j, w in enumerate(text_train[i]):
        # if j >= MAX_LEN:
        #     break       
        Y[i] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])
Y

tensor([45, 13,  1,  ..., 13,  1, 20])

In [46]:
Y.shape

torch.Size([31345])

In [47]:
# Сдвигаем на К = 3

X = Y.clone()
for j in range(X.shape[0]):    
        w = Y[j]
        if w != 0:  
            X[j] =  X[j] + 3
            if X[j] > 66:
                X[j] = X[j] - 66
X

tensor([48, 16,  4,  ..., 16,  4, 23])

In [48]:
X.shape

torch.Size([31345])

In [50]:
sentence = X[:21]
X = X[21:]

In [84]:
type(sentence)

torch.Tensor

In [79]:
class Network(torch.nn.Module):
    def __init__ (self):
        super(Network, self).__init__()
        self.embed = torch.nn.Embedding(len(X), 67)        
        self.rnn = torch.nn.RNN(67, 128)
        self.linear = nn.Linear(128, 67)
    def forward(self, X, state = None):
        embed = self.embed(X).view(1, len(X), 67)        
        o, s = self.rnn(embed)
        out = self.linear(o)
        return out

In [80]:
model = Network()

In [81]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

In [88]:
def generate_sentence(sentence = sentence):
    sentence = sentence
#     print(torch.tensor(sentence).view(1, 21, 1).shape)
    # sentence = [CHAR_TO_INDEX.get(s, 0) for s in sentence]    
    answers = model.forward(sentence)
    probas, indices = answers.topk(1)
    return " ".join([INDEX_TO_CHAR[ind.item()] for ind in indices.flatten()])

In [89]:
generate_sentence()

'Б ё Р п п м Щ Р х ё Я б Щ с ё б ф м Щ с б'

In [96]:
for ep in range(11):
    start = time.time()
    train_loss = 0.
    train_passed = 0   

    for i in range(int(len(X) / 100)):
        X_batch = X[i * 100:(i + 1) * 100]
    
        # X_batch = batch[:, :-1]
        embeding = nn.Embedding(len(X), 67)
        Y_batch = Y[21:][i * 100:(i + 1) * 100] #batch[:, 1:].flatten()
        
        
        optimizer.zero_grad()
        answers = model.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1       
        

    if ep%5 == 0:
        print("nEpoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
        s = generate_sentence()
        print(s)
    else:
        print(f"\rEpoch {ep}, loss: {train_loss / train_passed:.3f}", end='')    

nEpoch 0. Time: 7.714, Train loss: 0.001
К л а с с и к а д л я ш к о л ь н и к о в
Epoch 4, loss: 0.000nEpoch 5. Time: 8.200, Train loss: 0.000
К л а с с и к а д л я ш к о л ь н и к о в
Epoch 9, loss: 0.000nEpoch 10. Time: 7.822, Train loss: 0.000
К л а с с и к а д л я ш к о л ь н и к о в


#### Задание 2.
##### Выполнить практическую работу из лекционного ноутбука.
* а) построить RNN-ячейку на основе полносвязных слоев
* б) применить построенную ячейку для генерации текста с выражениями героев сериала “Симпсоны”

In [97]:
df2 = pd.read_csv('data.csv')
df2.head()

,Unnamed: 0,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
0,0,10368,35,29,"Lisa Simpson: Maggie, look. What's that?",235000,True,9,5.0,Lisa Simpson,Simpson Home,"Maggie, look. What's that?",maggie look whats that,4.0
1,1,10369,35,30,Lisa Simpson: Lee-mur. Lee-mur.,237000,True,9,5.0,Lisa Simpson,Simpson Home,Lee-mur. Lee-mur.,lee-mur lee-mur,2.0
2,2,10370,35,31,Lisa Simpson: Zee-boo. Zee-boo.,239000,True,9,5.0,Lisa Simpson,Simpson Home,Zee-boo. Zee-boo.,zee-boo zee-boo,2.0
3,3,10372,35,33,Lisa Simpson: I'm trying to teach Maggie that ...,245000,True,9,5.0,Lisa Simpson,Simpson Home,I'm trying to teach Maggie that nature doesn't...,im trying to teach maggie that nature doesnt e...,24.0
4,4,10374,35,35,"Lisa Simpson: It's like an ox, only it has a h...",254000,True,9,5.0,Lisa Simpson,Simpson Home,"It's like an ox, only it has a hump and a dewl...",its like an ox only it has a hump and a dewlap...,18.0


In [98]:
phrases2 = df2['normalized_text'].dropna().tolist()
phrases2[:10], len(phrases2)

(['maggie look whats that',
  'lee-mur lee-mur',
  'zee-boo zee-boo',
  'im trying to teach maggie that nature doesnt end with the barnyard i want her to have all the advantages that i didnt have',
  'its like an ox only it has a hump and a dewlap hump and dew-lap hump and dew-lap',
  'you know his blood type how romantic',
  'oh yeah whats my shoe size',
  'ring',
  'yes dad',
  'ooh look maggie what is that do-dec-ah-edron dodecahedron'],
 10891)

In [99]:
bag_words = []
for d in phrases2:
    bag_words.extend(d.split())
bag_words = [i for i in set(bag_words)]
CHARS2 = bag_words
len(CHARS2)

10849

In [100]:
text2 = [ph.split() for ph in phrases2 if type(ph) is str]

In [101]:
text2[:1]

[['maggie', 'look', 'whats', 'that']]

In [102]:
INDEX_TO_CHAR2 = ['none'] + [w for w in CHARS2]
CHAR_TO_INDEX2 = {w: i for i, w in enumerate(INDEX_TO_CHAR2)}

In [103]:
len(CHAR_TO_INDEX2)

10849

In [104]:
MAX_LEN2 = 75
X2 = torch.zeros((len(text2), MAX_LEN2), dtype=int)
for i in range(len(text2)):
    for j, w in enumerate(text2[i]):
        if j >= MAX_LEN2:
            break
        X2[i, j] = CHAR_TO_INDEX2.get(w, CHAR_TO_INDEX2['none'])

In [105]:
X2.shape

torch.Size([10891, 75])

In [106]:
L = 10849
H = 128
B = 100
E = 76

In [107]:
ht =  torch.zeros(1, B, H)
ht.shape 

torch.Size([1, 100, 128])

In [108]:
class Network2(torch.nn.Module):
    def __init__ (self):
        super(Network2, self).__init__()
        self.embed = torch.nn.Embedding(L, E)
        self.ln = torch.nn.Linear((H + E), H)
        self.tanh = torch.nn.Tanh()           
        self.linear = torch.nn.Linear(H, E)
        
    def forward(self, X2_batch, ht, state = None):
        r = torch.Tensor()
        embed = self.embed(X2_batch)
        
        for l in range(1, (embed.shape[1] + 1)):
            
            xt = embed[:,(l-1):l,:].view(1, B, E)
            xh = torch.cat([xt, ht], dim=2)
            ht = self.tanh(self.ln(xh))
            r = torch.cat([r, ht], dim=0) 
        
        out = self.linear(r)
        # print(out.shape)
        h = self.linear(ht)
        return out, h
model2 = Network2()
model2 

Network2(
  (embed): Embedding(10849, 76)
  (ln): Linear(in_features=204, out_features=128, bias=True)
  (tanh): Tanh()
  (linear): Linear(in_features=128, out_features=76, bias=True)
)

In [109]:
criterion2 = torch.nn.CrossEntropyLoss()
optimizer2 = torch.optim.SGD(model2.parameters(), lr=0.5)

In [110]:
def generate_sentence2():   
    sentence=  X2[-100:][:, :-1]    
    answers, h = model2.forward(sentence, ht)
    probas, indices = answers.topk(1)
    return " ".join([INDEX_TO_CHAR2[ind.item()] for ind in indices.flatten()])[:75]

In [111]:
generate_sentence2()

'they-they genre monsieur they-they discourage herself discourage excitement'

In [115]:
ht =  torch.zeros(1, B, H)
for ep in range(10):
    start = time.time()
    train_loss2 = 0.
    train_passed2 = 0

    for i in range(int(len(X2[:10000]) / 100)):
        
        batch = X2[:10000][i * 100:(i + 1) * 100]
        X2_batch = batch[:, :-1]
        
        embed = torch.nn.Embedding(L, E)       
        Y2_batch = embed(batch[:, 1:]).flatten().view(-1, 562400)       


        optimizer2.zero_grad()
        answers2, h = model2.forward(X2_batch, ht)
        answers3 = answers2.view(-1, 562400)      

        loss2 = criterion2(answers3, Y2_batch)
        train_loss2 += loss2.item()
        loss2.backward()
        optimizer2.step()
        train_passed2 += 1        

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss2 / train_passed2))
    print(generate_sentence2())   

Epoch 0. Time: 31.325, Train loss: -132159425167.360
died died their cans cans jailed cans without cans cans died died without w
Epoch 1. Time: 30.768, Train loss: 333290858823.680
died died glow died died jailed died without died cans without solved solve
Epoch 2. Time: 31.417, Train loss: 93331472691.200
prank died glow died died jailed died without died cans without without wit
Epoch 3. Time: 33.576, Train loss: 110179913963.520
prank signaling glow cans cans without cans died cans footprint died died d
Epoch 4. Time: 33.528, Train loss: -166934005882.880
signaling signaling monsieur cans cans hate cans died cans footprint died d
Epoch 5. Time: 32.183, Train loss: 249453889167.360
died leisure monsieur brown brown terrorist brown without brown they-they d
Epoch 6. Time: 33.640, Train loss: -136682133094.400
appointed appointed appointed chai chai terrorist chai without chai they-th
Epoch 7. Time: 30.207, Train loss: -874613586165.760
appointed appointed appointed cans cans terrorist